In [67]:
import pandas as pd
import polars as pl
pl.Config.set_tbl_cols(None)
pl.Config.set_tbl_rows(None)
pl.enable_string_cache()
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from sklearn.base import BaseEstimator, RegressorMixin
from collections import Counter
import gc


dataPath = "C:/Users/Marcel/Documents/Python/kaggle/CreditRiskModel/data/csv_files/"

In [68]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df



class Preprocess:

    def _numeric_readed_as_string(df: pl.DataFrame, col:str) -> pl.DataFrame:
        _missed_boolen = pl.Series(["false", 'true']).is_in(df.get_column(col).str.to_lowercase())
        if  _missed_boolen[0] == True or _missed_boolen[1] == True:
            df = df.with_columns(pl.col(col).str.to_lowercase().str.replace('false',False).str.replace('true', True).cast(pl.Float64).alias(col))
        if col[-1] in ["D"] or col in ["date_decision"]:
            df = df.with_columns(pl.col(col).str.replace('"','').cast(pl.Date).alias(col))
        else:
            df = df.with_columns(pl.col(col).str.replace('"','').cast(pl.Float64).alias(col))
        return df
    
    # needed cause set_table_dtypes returns 4 list of columns (bad design)
    def _devide_into_type_categories(copy_dtypes: list) -> list:
        numeric_col =[]
        timedue_col = []
        string_col = []
        date_col = []
        list_names, list_dtypes = copy_dtypes
        for name, col_type in zip(list_names,list_dtypes):
            if name[-1] in ("P", "A"): 
                numeric_col.append(name)
            elif name[-1] in ("T"):
                if col_type == pl.Int64:
                    timedue_col.append(name)
                else:
                     string_col.append(name)
            elif name[-1] in ("L"):
                if col_type == pl.Float64:
                    numeric_col.append(name)
                else:
                    string_col.append(name)
            elif name[-1] in ("D") or name in ["date_decision"]:
                date_col.append(name)
            elif name[-1] in ("M"):
                string_col.append(name)
        return [numeric_col,timedue_col,string_col,date_col]
            

    def _paste_dtypes(df: pl.DataFrame, copy_dtypes: tuple) -> pl.DataFrame:
        list_names, list_dtypes = copy_dtypes
        df = df.select(list_names)
        for name, col_type in zip(list_names,list_dtypes):
            try:
                if df.get_column(name).dtype in [pl.Utf8]:
                    df = Preprocess._numeric_readed_as_string(df, name)
            except:
                pass
            df = df.with_columns(pl.col(name).cast(col_type).alias(name))
        return df , Preprocess._devide_into_type_categories(copy_dtypes)

    def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
        numeric_col =[]
        timedue_col = []
        string_col = []
        date_col = []
        for col in df.columns:
            if col.strip() in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64).alias(col))
            elif col[-1] in ("P", "A"):
                if df.get_column(col).dtype in [pl.Utf8]:
                       # for instance "False", or '"2.0"'
                       df = Preprocess._numeric_readed_as_string(df, col)
                df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
                numeric_col.append(col)
            elif col[-1] in ("T"):
                try:
                    if df.get_column(col).dtype in [pl.Utf8]:
                       # for instance "False", or '"2.0"'
                       df = Preprocess._numeric_readed_as_string(df, col)
                    df = df.with_columns(pl.col(col).cast(pl.Int64).alias(col))
                    timedue_col.append(col)
                except:
                     df = df.with_columns(pl.col(col).cast(pl.Utf8).alias(col))
                     string_col.append(col)
            elif col[-1] in ("L"):
                try:
                    if df.get_column(col).dtype in [pl.Utf8]:
                       # for instance "False", or '"2.0"'
                       df = Preprocess._numeric_readed_as_string(df, col)
                    df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
                    numeric_col.append(col)
                except:
                    df = df.with_columns(pl.col(col).cast(pl.Utf8).alias(col))
                    string_col.append(col)
            elif col[-1] in ("D") or col in ["date_decision"]:
                if df.get_column(col).dtype in [pl.Utf8]:
                       # for instance "False", or '"2.0"'
                       df = Preprocess._numeric_readed_as_string(df, col)
                df = df.with_columns(pl.col(col).cast(pl.Date).alias(col))
                date_col.append(col)
            elif col[-1] in ("M"):
                df = df.with_columns(pl.col(col).cast(pl.Utf8).alias(col))
                string_col.append(col)
        return df, [numeric_col,timedue_col,string_col,date_col]
    
    def offset_dates_by_decision_date(df: pl.DataFrame) -> pl.DataFrame:
        decision_data = df.get_column("date_decision")
        for col in df:
            if col.name[-1] in ["D"]:
                df = df.with_columns((col - pl.col("date_decision")).dt.days().cast(pl.Int64).alias(col.name))
        df = df.drop("date_decision","MONTH")
        return df
    
    def remove_columns_over_null_limit(df: pl.DataFrame) -> pl.DataFrame:
        drop_list = []
        for col in df:
            if (col.name not in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]) and (col.null_count()/len(col) > 0.7):
                drop_list.append(col.name)
        df = df.drop(drop_list)
        return df, drop_list
    
    def remove_columns_over_max_cardinality(df: pl.DataFrame) -> pl.DataFrame:
        drop_list = []
        for col in df:
            if (col.name not in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]) and (col.dtype == pl.Utf8) and (col.n_unique() > 200):
                drop_list.append(col.name)
        df = df.drop(drop_list)
        return df, drop_list



In [69]:
class Agregator:

    def aggreget(df: pl.DataFrame, column_types) -> pl.DataFrame:
        numeric_col, timedue_col, string_col, date_col = column_types
        
        df = df.group_by('case_id').agg(pl.col(numeric_col).sum(), pl.col(timedue_col).mean(),pl.col(string_col).mode(),pl.col(date_col).mean())
        for col in string_col:
            df = df.with_columns(pl.col(col).list.first().alias(col))

        return df

In [70]:
class Loader:
    def _count_files_for_concat(all_files: list) -> dict:
        striped_names = []
        for file_name in all_files:
            suffix = file_name.split('_')[-1]
            striped_names.append(file_name.strip(suffix))
        count =  Counter(striped_names)
        return count

    def _column_types_names(files: list) -> list:
        files_colname_type = []
        for file in files:
            column_name = []
            column_type = []
            for column in file:
                if column.name not in ['target']:
                    column_name.append(column.name)
                    column_type.append(column.dtype)
            files_colname_type.append((column_name,column_type))
        return files_colname_type
    

    def load_data_by_depth(paths_list: list, depth: int, copy_load: list = None) -> list:
        files_list = []
        files_count = Loader._count_files_for_concat(paths_list)
        total_path_index = 0
        file_number = 0
        # loop throught distinct files names and their amount
        for file_name, count in files_count.items():
            temp_storage_for_concat = []
            #read all files which has the same name
            for idx in range(count):
                file_path = dataPath + paths_list[total_path_index].split('_',2)[0] + '/' + paths_list[total_path_index]
                if copy_load == None:
                    file, column_types = Preprocess.set_table_dtypes(pl.read_csv(file_path))
                else:
                    file, column_types = Preprocess._paste_dtypes(pl.read_csv(file_path),copy_load[file_number])
                # there are empty files:
                if len(file) == 0: 
                    total_path_index += 1
                    continue

                if depth > 0 :
                    file = Agregator.aggreget(file, column_types)
                temp_storage_for_concat.append(file)
                total_path_index += 1
            file_number +=1
            if temp_storage_for_concat:
                file = pl.concat(temp_storage_for_concat, how="vertical_relaxed")
                file,_ = Preprocess.remove_columns_over_null_limit(file)
                file,_ = Preprocess.remove_columns_over_max_cardinality(file)
                files_list.append(file)
            gc.collect()
        return files_list

    def load_files(train_paths_dictionary: dict, test_paths_dictionary: dict ) -> list:
        train_files = []
        test_files = []
        for depth in train_paths_dictionary:
            train = Loader.load_data_by_depth(train_paths_dictionary[depth], depth)
            train_files.append(train)
            column_name_type = Loader._column_types_names(train)
            test_files.append(Loader.load_data_by_depth(test_paths_dictionary[depth], depth, column_name_type))
            print('loaded ', depth)
        return train_files,test_files
    
    def join_tables(list_of_files: list, base_table: pl.DataFrame) -> pl.DataFrame:
        for files_on_given_deapth in list_of_files:
            for file in files_on_given_deapth:

                base_table = base_table.join(
                    file, how="left", on="case_id")
        return base_table
    
    def choose_columns_by_availability(train_columns: list, test_columns: list) -> list:
        columns = list(set(train_columns) & set(test_columns))
        return columns, columns
    

In [71]:
data_path_train = {0: ["train_static_0_0.csv","train_static_0_1.csv","train_static_cb_0.csv"],
                   1: ["train_applprev_1_0.csv","train_applprev_1_1.csv","train_other_1.csv","train_tax_registry_a_1.csv", "train_tax_registry_b_1.csv","train_tax_registry_c_1.csv",
                              "train_credit_bureau_a_1_0.csv","train_credit_bureau_a_1_1.csv","train_credit_bureau_a_1_2.csv","train_credit_bureau_a_1_3.csv","train_credit_bureau_b_1.csv",
                              "train_deposit_1.csv","train_person_1.csv","train_debitcard_1.csv"],
                    2: ["train_applprev_2.csv","train_person_2.csv","train_credit_bureau_a_2_0.csv","train_credit_bureau_a_2_1.csv","train_credit_bureau_a_2_2.csv","train_credit_bureau_a_2_3.csv",
                             "train_credit_bureau_a_2_4.csv","train_credit_bureau_a_2_5.csv","train_credit_bureau_a_2_6.csv","train_credit_bureau_a_2_7.csv","train_credit_bureau_a_2_8.csv","train_credit_bureau_a_2_9.csv",
                             "train_credit_bureau_a_2_10.csv","train_credit_bureau_b_2.csv"]}

data_path_test = {0: ["test_static_0_0.csv","test_static_0_1.csv","test_static_0_2.csv","test_static_cb_0.csv"],
                  1:  ["test_applprev_1_0.csv","test_applprev_1_1.csv","test_applprev_1_2.csv", "test_other_1.csv", "test_tax_registry_a_1.csv","test_tax_registry_b_1.csv","test_tax_registry_c_1.csv",
                              "test_credit_bureau_a_1_0.csv","test_credit_bureau_a_1_1.csv","test_credit_bureau_a_1_2.csv","test_credit_bureau_a_1_3.csv","test_credit_bureau_a_1_4.csv","test_credit_bureau_b_1.csv",
                              "test_deposit_1.csv","test_person_1.csv","test_debitcard_1.csv"],
                  2: ["test_applprev_2.csv","test_person_2.csv","test_credit_bureau_a_2_0.csv","test_credit_bureau_a_2_1.csv","test_credit_bureau_a_2_2.csv","test_credit_bureau_a_2_3.csv",
                             "test_credit_bureau_a_2_4.csv","test_credit_bureau_a_2_5.csv","test_credit_bureau_a_2_6.csv","test_credit_bureau_a_2_7.csv","test_credit_bureau_a_2_8.csv","test_credit_bureau_a_2_9.csv",
                             "test_credit_bureau_a_2_10.csv","test_credit_bureau_a_2_11.csv","test_credit_bureau_b_2.csv"]}


In [72]:
#load_files returns list for every deapth of files it reads so it has 2 layers of list on top of the pl.dataframe
train_basetable, test_basetable = Loader.load_files({0: ["train_base.csv"]},{0: ["test_base.csv"]})
train_files, test_files = Loader.load_files(data_path_train, data_path_test)
train_joined = Loader.join_tables(train_files, train_basetable[0][0])
test_joined = Loader.join_tables(test_files, test_basetable[0][0])
del train_files, test_files, train_basetable, test_basetable
gc.collect()
train_joined,delete_list = Preprocess.remove_columns_over_null_limit(train_joined)
test_joined  = test_joined.drop(delete_list)
train_joined = train_joined.pipe(Preprocess.offset_dates_by_decision_date)
test_joined = test_joined.pipe(Preprocess.offset_dates_by_decision_date)

loaded  0
loaded  0
loaded  1
loaded  2


In [73]:
train_columns, test_columns = Loader.choose_columns_by_availability(train_joined.columns, test_joined.columns)

In [41]:
print(f'Train data size: {train_joined.estimated_size()/1024**2}MB    Test data size: {test_joined.estimated_size()/1024**2}MB')


Train data size: 4083.6790161132812MB    Test data size: 0.024745941162109375MB


## Feature engineering

In this part, we can see a simple example of joining tables via `case_id`. Here the loading and joining is done with polars library. Polars library is blazingly fast and has much smaller memory footprint than pandas. 

In [75]:
case_ids = pd.DataFrame(train_joined["case_id"].unique(), columns=['case_id'])
case_ids_train, case_ids_valid = train_test_split(case_ids, train_size=0.8, random_state=1, shuffle=False)
case_ids_train = case_ids_train.to_numpy().reshape(-1)
case_ids_valid = case_ids_valid.to_numpy().reshape(-1)


def from_polars_to_pandas(df,case_ids: pl.Series, columns_filter) -> pl.DataFrame:
    return (
        df.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        df.filter(pl.col("case_id").is_in(case_ids))[columns_filter].to_pandas(),
        df.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )


test_joined = test_joined.to_pandas()

base_train, X_train, y_train = from_polars_to_pandas(train_joined,case_ids_train,train_columns)
base_valid, X_valid, y_valid = from_polars_to_pandas(train_joined,case_ids_valid,train_columns)
for df in [X_train,X_valid, test_joined]:
    df = convert_strings(df)

In [62]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {test_joined.shape}")

Train: (1221327, 286)
Valid: (305332, 286)
Test: (10, 286)


## Training LightGBM

Minimal example of LightGBM training is shown below.

In [76]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

C:\Users\Marcel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.745614
[100]	valid_0's auc: 0.769889
[150]	valid_0's auc: 0.781988
[200]	valid_0's auc: 0.788971
[250]	valid_0's auc: 0.792884
[300]	valid_0's auc: 0.797546
[350]	valid_0's auc: 0.801184
[400]	valid_0's auc: 0.803905
[450]	valid_0's auc: 0.805291
Early stopping, best iteration is:
[463]	valid_0's auc: 0.806628


Evaluation with AUC and then comparison with the stability metric is shown below.

In [77]:
for base, X in [(base_train, X_train), (base_valid, X_valid)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 


The AUC score on the train set is: 0.8391168516079585
The AUC score on the valid set is: 0.8066282100832387


In [82]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std, a, avg_gini, res_std

stability_score_train, a_train, avg_gini_train, res_std_train = gini_stability(base_train)
stability_score_valid, a_valid, avg_gini_valid, res_std_valid = gini_stability(base_valid)


print(f'The stability score on the train set is: {stability_score_train} a = {a_train} avg_gini = {avg_gini_train} res_std = {res_std_train}') 
print(f'The stability score on the valid set is: {stability_score_valid} a = {a_valid} avg_gini = {avg_gini_valid} res_std = {res_std_valid}') 


The stability score on the train set is: 0.6424208964868618 a = 0.00011886247492601548 avg_gini = 0.6644752351829734 res_std = 0.044108677392222975
The stability score on the valid set is: 0.54664618678408 a = 0.003252229519540812 avg_gini = 0.5790713352144955 res_std = 0.06485029686083121


## Submission

Scoring the submission dataset is below, we need to take care of new categories. Then we save the score as a last step. 

In [80]:
X_submission = test_joined[train_columns]
X_submission = convert_strings(X_submission)

y_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration)

In [81]:
submission = pd.DataFrame({
    "case_id": test_joined["case_id"].to_numpy(),
    "score": y_submission_pred
}).set_index('case_id')
submission.to_csv("./submission.csv")